``` SQL
ВОПРОС 1

SELECT *
FROM Клиенты
WHERE Клиент_ФИО LIKE 'Иванов %' AND EXTRACT(YEAR FROM Клиент_рождение::date) >= 2000

ВОПРОС 2

WITH bad_client as (
SELECT Клиент_ФИО
FROM Счета
WHERE Счет_Тип = 'Пенсионный'),

filt_df as (
SELECT *
FROM Счета
WHERE Клиент_ФИО NOT IN bad_client AND Счет_Тип = 'Сберегательный')

SELECT DISTINCT Клиент_ФИО,
       COUNT(Счет_Номер) OVER n_window AS cnt_acc,
       MIN(Счет_ДатаОткрытия::date) OVER n_window AS min_dt,
       MAX(Счет_ДатаОткрытия::date) OVER n_windoW AS min_dt
FROM filt_df
WINDOW (PARTITION BY Клиент_ФИО)

ВОПРОС 3

WITH df AS (
SELECT Счет_Номер, Месяц_Номер, SUM(Пополнения_ИтогСумма) AS fin_sum
FROM "История Операций"
WHERE Месяц_Номер = 9 OR Месяц_Номер = 10
GROUP BY Счет_Номер, Месяц_Номер),

df1 as (
SELECT Счет_Номер, fin_sum as sum_sep
FROM df
WHERE Месяц_Номер = 9),

df2 as (
SELECT Счет_Номер, fin_sum as sum_oct
FROM df
WHERE Месяц_Номер = 10),

fin_df as ( 
SELECT df1.Счет_Номер, 
       ISNULL(sum_oct, 0) as sum_oct, 
       ISNULL(sum_sep, 0) as sum_sep, 
       sum_oct - sum_sep as diff
FROM df1 OUTER JOIN df2 ON df1.Счет_Номер=df2.Счет_Номер)

SELECT Счет_Номер, sum_oct, sum_sep
FROM fin_df
WHERE diff > 0
```

In [1]:
text = 'Мама УТРом шить Маша шитье забыть ШитЬЕ прошИть ШапкА ниша положить выше Мама утроМ Шапка стужа МашА вышла вечЕР пуЩе СтуЖА ШашКа положитЬ луЖА нИша ШИть стуЖа вЫшЕ Ниша стУЖа УтроМ маша'

In [8]:
dict1 = []
for w in text.split():
    if ('Ш' in w) or ('ш' in w):
        dict1.append(w.lower())
    

In [10]:
import pandas as pd

In [20]:
df = pd.DataFrame(dict1, columns=['word'])

In [21]:
df['count'] = 1

In [22]:
df = df.groupby('word')['count'].sum().reset_index()

In [27]:
df[df['count'] >= 2]

,word,count
0,выше,2
2,маша,3
3,ниша,3
5,шапка,2
7,шить,2
8,шитье,2
